<h3> Laboratorio N 5 </h3>

Nombre de integrantes del grupo:
    
    1. Diego Irarrazaval
     
Deben subir el resultado del grupo en un jupyter notebook hasta el día Martes 7 de Diciembre a las 23:00 horas!!

El laboratorio consta de 5 preguntas donde la nota del laboratorio corresponde a la suma del puntaje más un punto base.

----

<h3>Pregunta 1. </h3> (0.5 pts)

Haga el de tratamiento de variables así como de pre procesamiento de datos para obtener un dataframe que le permita correr algoritmos de clasificación como los estudiados en el auxiliar 5.

Debe formular el dataset como un problema de clasificación binario sobre la columna ´SaleFlag´



In [1]:
import pandas as pd

In [2]:
housing = pd.read_csv('https://raw.githubusercontent.com/saguerraty/Aux-IN6531-/master/Lab%205/housing_categorical.csv')

In [3]:
try:
  housing.drop(labels=['SaleCondition', 'Unnamed: 0'], axis=1, inplace=True)
except:
  pass

housing.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageCars,GarageArea,OpenPorchSF,PoolArea,PoolQC,Fence,MoSold,YrSold,SaleFlag
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,1Fam,2Story,7,5,2003,2003,Gable,Gd,TA,PConc,Gd,TA,706,150,856,GasA,Ex,Y,SBrkr,856,854,1710,2,1,3,1,Gd,8,0,NaN,2,548,61,0,NaN,NaN,2,2008,Normal
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,1Fam,1Story,6,8,1976,1976,Gable,TA,TA,CBlock,Gd,TA,978,284,1262,GasA,Ex,Y,SBrkr,1262,0,1262,2,0,3,1,TA,6,1,TA,2,460,0,0,NaN,NaN,5,2007,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,1Fam,2Story,7,5,2001,2002,Gable,Gd,TA,PConc,Gd,TA,486,434,920,GasA,Ex,Y,SBrkr,920,866,1786,2,1,3,1,Gd,6,1,TA,2,608,42,0,NaN,NaN,9,2008,Normal
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,1Fam,2Story,7,5,1915,1970,Gable,TA,TA,BrkTil,TA,Gd,216,540,756,GasA,Gd,Y,SBrkr,961,756,1717,1,0,3,1,Gd,7,1,Gd,3,642,35,0,NaN,NaN,2,2006,Abnorml
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,1Fam,2Story,8,5,2000,2000,Gable,Gd,TA,PConc,Gd,TA,655,490,1145,GasA,Ex,Y,SBrkr,1145,1053,2198,2,1,4,1,Gd,9,1,TA,3,836,84,0,NaN,NaN,12,2008,Normal


In [4]:
numeric_vars = ['LotArea', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF',
                '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'FullBath',
                'HalfBath', 'BedroomAbvGr','KitchenAbvGr',  'TotRmsAbvGrd', 
                'Fireplaces','GarageCars','GarageArea', 'OpenPorchSF', 'PoolArea']
categorical_vars = ['MSSubClass', 'MSZoning', 'Street', 'LotShape', 
                    'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
                    'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond',
                    'RoofStyle', 'ExterQual', 'ExterCond', 'Foundation',
                    'BsmtQual', 'BsmtCond', 'Heating', 'HeatingQC', 'CentralAir',
                    'Electrical', 'KitchenQual']
other = ['YearBuilt', 'YearRemodAdd', 'MoSold', 'YrSold']
target = 'SaleFlag'

In [5]:
housing.columns[housing.isnull().any()]

Index(['LotFrontage', 'BsmtQual', 'BsmtCond', 'Electrical', 'FireplaceQu',
       'PoolQC', 'Fence'],
      dtype='object')

In [6]:
housing.isnull().sum()

MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
Street             0
LotShape           0
LandContour        0
Utilities          0
LotConfig          0
LandSlope          0
BldgType           0
HouseStyle         0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
RoofStyle          0
ExterQual          0
ExterCond          0
Foundation         0
BsmtQual          37
BsmtCond          37
BsmtFinSF1         0
BsmtUnfSF          0
TotalBsmtSF        0
Heating            0
HeatingQC          0
CentralAir         0
Electrical         1
1stFlrSF           0
2ndFlrSF           0
GrLivArea          0
FullBath           0
HalfBath           0
BedroomAbvGr       0
KitchenAbvGr       0
KitchenQual        0
TotRmsAbvGrd       0
Fireplaces         0
FireplaceQu      690
GarageCars         0
GarageArea         0
OpenPorchSF        0
PoolArea           0
PoolQC          1453
Fence           1179
MoSold             0
YrSold       

Se deben rellenar las variables que se muestran debido a que hay una cantidad considerable de datos faltantes. 
De la documentacion del dataset, se observa que las variables que tienen missing values son el siguiente tipo de datos (ejemplo de `FireplaceQu`):
```
FireplaceQu: Fireplace quality

       Ex	Excellent - Exceptional Masonry Fireplace
       Gd	Good - Masonry Fireplace in main level
       TA	Average - Prefabricated Fireplace in main living area or Masonry Fireplace in basement
       Fa	Fair - Prefabricated Fireplace in basement
       Po	Poor - Ben Franklin Stove
       NA	No Fireplace
```
Por lo que los missing values se rellenaran con `NA`, a excepcion de `LotFrontage`, que es una variable numerica que se rellenara con `0` y electrical en donde se eliminara la fila que no contiene informacion. 

In [7]:
missing_values_cols = ['BsmtQual', 'BsmtCond', 'PoolQC', 'Fence', 'FireplaceQu']

for col in missing_values_cols:
  housing[col].fillna(value='NA', inplace=True)

housing['LotFrontage'].fillna(value=0, inplace=True)

In [8]:
# Chequeamos que solo haya missing values en la columna 'electrical':
housing.columns[housing.isnull().any()]

Index(['Electrical'], dtype='object')

In [9]:
# botemos esa la fila correspondiente al missing value:
housing.dropna(axis=0, how='any', inplace=True)

In [10]:
# Chequeamos que no haya missing values:
housing.columns[housing.isnull().any()]

Index([], dtype='object')

In [11]:
#Preprocesamiento corresponde en transformar las variables que son texto a categoricas. 

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, PolynomialFeatures, LabelEncoder
from sklearn.impute import SimpleImputer
import numpy as np
# pipelines
cat_pipe = Pipeline(steps=[('encode', OneHotEncoder(sparse=False))])
std_pipe = Pipeline(steps=[('std_transform', StandardScaler())])
poly_pipe = Pipeline(steps=[('poly', PolynomialFeatures(3))])


# ¡Transformers, más de lo que ves!
col_transformer = ColumnTransformer(transformers=[('cat', cat_pipe, categorical_vars),
                                                  #('poly', poly_pipe, numeric_vars),
                                                  ('minmax', std_pipe, numeric_vars)])

In [12]:
# vamos a crear una copia del df para tener uno origal:
data = housing.copy()
X = col_transformer.fit_transform(data.drop(labels='SaleFlag',axis=1))

In [13]:
# Debemos transformar tambien la columna target:
le = LabelEncoder()
y = le.fit_transform(data[target])

<h3>Pregunta 2.</h3> (0.5 pts)

Explique la diferencia entre las metricas Accuracy, Precision, Recall y ROC_AUC

Ademas defina las métricas que utilizará para evaluar el rendimiento del modelo que entrene para predecir la variable ´SaleFlag´, puede definir de manera explicita supuestos que le ayuden a justificar la elección de las métricas seleccionadas.



In [14]:
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, f1_score, classification_report, accuracy_score, recall_score, precision_score, confusion_matrix
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(8,8))
    plt.title(title)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm = np.round(cm, decimals=3)

        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')


    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.imshow(cm, interpolation='nearest', cmap=cmap)

    plt.colorbar()

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

<h3>Pregunta 3.</h3> (1.5 pts)

Sin modificar el dataset luego del procesamiento, entrene 5 algoritmos distinos de clasificación que le permitan determinar la variable ´SaleFlag´.
Indique si está entrenando un problema balanceado y en que proporción está cada etiqueta.

Determine y reporte el algoritmo que logra predecir la etiqueta de mejor manera según los criterios definidos en la pregunta 2. Justifique su elección.


In [15]:
# obtengamos el la cantidad de elementos por clase:
unique, counts = np.unique(y, return_counts=True)
clases = dict(zip(unique, counts))
print(clases)

{0: 262, 1: 1197}


In [16]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, pd.DataFrame(y, columns=['SaleFlag']), test_size=0.2, random_state=42)

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF


models = []
models.append(('LogisticRegression  ', LogisticRegression(class_weight = clases)))
models.append(('KNN    ', KNeighborsClassifier()))
models.append(('GaussianProcessClassifier ', GaussianProcessClassifier(kernel = 1.0 * RBF(1.0))))
models.append(('RandomForest ', RandomForestClassifier()))
models.append(('SVM    ', SVC()))

In [18]:
  for name, model in models:
    #Ajuste del modelo
    try:
      model.fit(X_train, y_train)
      #Predecir
      y_pred = model.predict(X_test)
      #metricas de conjunto de entrenamiento
      train_pred = model.predict(X_train)
    except TypeError:
      model.fit(X_train.todense(), y_train)  
      #Predecir
      y_pred = model.predict(X_test.todense())
      #metricas de conjunto de entrenamiento
      train_pred = model.predict(X_train.todense())
  
    
    #Metricas de testeo
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall  = recall_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    

    train_acc = accuracy_score(y_train, train_pred)
    train_precision = precision_score(y_train, train_pred)
    train_recall = recall_score(y_train, train_pred)
    #Print de datos
    
    print("Modelo: {}".format(name))
    print('Train accuracy: {:.2f}, train precision: {:.2f}, train recall: {:.2f}'.format(train_acc, train_precision, train_recall))
    print("Test accuracy: {:.2f}, Test precision: {:.2f}, Test recall: {:.2f}, Test AUC: {:.2f}".format(accuracy, precision, recall, roc_auc))
    print("")

Modelo: LogisticRegression  
Train accuracy: 0.83, train precision: 0.82, train recall: 1.00
Test accuracy: 0.84, Test precision: 0.85, Test recall: 0.99, Test AUC: 0.54



/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

Modelo: KNN    
Train accuracy: 0.84, train precision: 0.85, train recall: 0.97
Test accuracy: 0.81, Test precision: 0.84, Test recall: 0.95, Test AUC: 0.52



/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Modelo: GaussianProcessClassifier 
Train accuracy: 0.82, train precision: 0.82, train recall: 1.00
Test accuracy: 0.84, Test precision: 0.84, Test recall: 1.00, Test AUC: 0.50



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


Modelo: RandomForest 
Train accuracy: 1.00, train precision: 1.00, train recall: 1.00
Test accuracy: 0.84, Test precision: 0.85, Test recall: 0.99, Test AUC: 0.54



/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Modelo: SVM    
Train accuracy: 0.84, train precision: 0.83, train recall: 1.00
Test accuracy: 0.83, Test precision: 0.84, Test recall: 1.00, Test AUC: 0.50



<h3>Pregunta 4. </h3> (1 pto)

Aplique las técnicas de sobre y sub sampleo para balancear el dataset de manera equitativa, entrene los mismos 5 algoritmos de la pregunta 3 usando el dataset balnceado de manera respectiva.

Para cada técnica determine y reporte el algoritmo que logra predecir la etiqueta de mejor manera según los criterios definidos en la pregunta 2. Justifique su elección.

In [29]:
from sklearn.utils import resample

X_train, X_test, y_train, y_test = train_test_split(data.drop(labels='SaleFlag', axis=1), data['SaleFlag'], test_size=0.2, random_state=42)

with_class_df = pd.concat([pd.DataFrame(X_train), y_train], axis=1)

df_majority = with_class_df[ with_class_df['SaleFlag'] == 'Normal']
df_minority = with_class_df[ with_class_df['SaleFlag'] == 'Abnorml']

malig_upsampled = resample( df_minority, replace=True, n_samples= len(df_majority) ,random_state=123)

upsampled_train = pd.concat([malig_upsampled, df_majority], axis=0)

unique, counts = np.unique(upsampled_train['SaleFlag'], return_counts=True)
clases = dict(zip(unique, counts))
print(clases)

{'Abnorml': 953, 'Normal': 953}


In [30]:
col_transformer = ColumnTransformer(transformers=[('cat', cat_pipe, categorical_vars),
                                                  #('poly', poly_pipe, numeric_vars),
                                                  ('minmax', std_pipe, numeric_vars)])

X_train = col_transformer.fit_transform(upsampled_train.drop(labels='SaleFlag',axis=1))
X_test = col_transformer.fit_transform(X_test)

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

In [31]:
models = []
models.append(('LogisticRegression  ', LogisticRegression(class_weight = clases)))
models.append(('KNN    ', KNeighborsClassifier()))
models.append(('GaussianProcessClassifier ', GaussianProcessClassifier(kernel = 1.0 * RBF(1.0))))
models.append(('RandomForest ', RandomForestClassifier()))
models.append(('SVM    ', SVC()))

X_train = upsampled_train
y_train = upsampled_train['SaleFlag']

for name, model in models:
    #Ajuste del modelo
    try:
      model.fit(X_train, y_train)
      #Predecir
      y_pred = model.predict(X_test)
      #metricas de conjunto de entrenamiento
      train_pred = model.predict(X_train)
    except TypeError:
      model.fit(X_train.todense(), y_train)  
      #Predecir
      y_pred = model.predict(X_test.todense())
      #metricas de conjunto de entrenamiento
      train_pred = model.predict(X_train.todense())
  
    
    #Metricas de testeo
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall  = recall_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    

    train_acc = accuracy_score(y_train, train_pred)
    train_precision = precision_score(y_train, train_pred)
    train_recall = recall_score(y_train, train_pred)
    #Print de datos
    
    print("Modelo: {}".format(name))
    print('Train accuracy: {:.2f}, train precision: {:.2f}, train recall: {:.2f}'.format(train_acc, train_precision, train_recall))
    print("Test accuracy: {:.2f}, Test precision: {:.2f}, Test recall: {:.2f}, Test AUC: {:.2f}".format(accuracy, precision, recall, roc_auc))
    print("")

ValueError: ignored



<h3>Pregunta 5.</h3>  (1.5 pts)

Utilice el método de 'Synthetic Minority Oversampling Technique' para balancear el dataset de manera equitativa y con un 60% de las etiquetas de la clase dominante original.

Para cada caso según corresponda, entrene los mismos 5 algoritmos utilizados para la pregunta 3 para cada versión del dataset balanceado con SMOTE.

Para cada técnica determine y reporte el algoritmo que logra predecir la etiqueta de mejor manera según los criterios definidos en la pregunta 2. Justifique su elección.

Además de todos los modelos y configuraciones probadas en las preguntas 3, 4 y 5 determine que modelo le entrega el mejor rendimiento para predecir la variable objetivo y comente el resultado encontrado.

<h3> Pregunta 6. </h3> (1 pto)

Aplique la técnica de validación cruzada al mejor modelo que ha entrenado para este problema y determine si su modelo es robusto a la variabilidad de la seleción del conjunto de entrenamiento, para esto utilice las mismas métricas definidas en la pregunta 2.

Hint: puede utilizar la función ennumerate() de python y cross_validate() del modulo 'sklearn.model_selection', si tiene dudas pregunte.